<a href="https://colab.research.google.com/github/artyomsol23/google_colab/blob/main/my_homework_7_week%5Breu_ml_school%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**ML School by REU Data Scinece Club**</center>

----
## <center> Неделя №7 </center>

На этой неделе вам предстоит написать Adaboost, основанный на Decision tree. Это послдений алгоритм, который мы будем писать сами

Импортируем нужные библиотеки

In [5]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

Загрузим данные, мы будем пытаться предсказать космические тела - галактики и звезды(колонка class). Что обозначает каждая колонка нам не особо важно также как и класс. Данные уже обработаны. Если вам интересно почитать о них, то вот [ссылка](https://www.sdss.org/dr18/) на официальный сайт

In [6]:
!git clone https://github.com/MeinRespect/REU-Ml-SCHOOL.git

fatal: destination path 'REU-Ml-SCHOOL' already exists and is not an empty directory.


In [7]:
df = pd.read_csv('/content/REU-Ml-SCHOOL/galaxy_data.csv').drop('objid', axis=1)
df.sample(10)

,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
87072,126.682462,40.735535,19.54835,18.40678,18.00699,17.88169,17.80872,2825,301,6,146,3550105972905190000,STAR,0.000125,3153,54790,522
50495,116.978381,83.529319,18.09425,16.82281,16.85647,16.96203,17.05081,5960,301,3,185,2861007121099550000,STAR,-0.000274,2541,54481,347
27633,212.703546,2.579010,19.04782,17.30737,16.34563,15.91753,15.48381,1458,301,4,587,599132715505183000,GALAXY,0.118643,532,51993,560
99368,148.958844,19.367043,19.06222,17.21920,16.27533,15.85125,15.50485,5137,301,1,237,2669614295356040000,GALAXY,0.090001,2371,53762,384
42219,136.283306,31.318844,19.03929,17.87254,17.38067,17.03610,16.84484,3704,301,4,36,1791373047291210000,GALAXY,0.062493,1591,52976,241
52463,223.458012,28.106974,19.51209,17.77952,16.85442,16.42610,16.09900,4570,301,2,104,2416242019069360000,GALAXY,0.126913,2146,54230,221
53948,234.930404,37.609017,19.26476,18.01357,17.52307,17.20367,17.01603,3958,301,4,35,1456931293194380000,GALAXY,0.076557,1294,52753,61
93724,326.311123,0.777727,18.74769,17.83378,17.43596,17.25785,17.16480,7717,301,1,104,4724474013111570000,STAR,-0.001668,4196,55478,720
23382,165.191274,57.236817,18.34997,18.16349,18.01111,18.07979,18.20549,2243,301,1,266,1068480701579420000,QSO,1.139861,949,52427,6
86160,207.077173,7.627383,19.45512,17.77196,16.88866,16.36077,15.98896,3905,301,5,49,2032339007060340000,GALAXY,0.104062,1805,53875,326


В наших данных три класса. Сейчас мы не будем решать задачу мультиклассовой классификации, поэтому просто удалим 1 класс и изменим обозначения на -1 и 1

In [8]:
df['class'].value_counts()

,count
class,
GALAXY,51323
STAR,38096
QSO,10581


In [9]:
df = df[df['class'].isin(['GALAXY', 'STAR'])]
df['class'].replace({'GALAXY':-1, 'STAR':1}, inplace=True)
df

/tmp/ipython-input-9-1532312225.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['class'].replace({'GALAXY':-1, 'STAR':1}, inplace=True)
/tmp/ipython-input-9-1532312225.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['class'].replace({'GALAXY':-1, 'STAR':1}, inplace=True)
/tmp/ipython-i

,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,47.372545,0.820621,18.69254,17.13867,16.55555,16.34662,16.17639,4849,301,5,771,8168632633242440000,1,0.000115,7255,56597,832
1,116.303083,42.455980,18.47633,17.30546,17.24116,17.32780,17.37114,6573,301,6,220,9333948945297330000,1,-0.000093,8290,57364,868
2,172.756623,-8.785698,16.47714,15.31072,15.55971,15.72207,15.82471,5973,301,1,13,3221211255238850000,1,0.000165,2861,54583,42
3,201.224207,28.771290,18.63561,16.88346,16.09825,15.70987,15.43491,4649,301,3,121,2254061292459420000,-1,0.058155,2002,53471,35
4,212.817222,26.625225,18.88325,17.87948,17.47037,17.17441,17.05235,4649,301,3,191,2390305906828010000,-1,0.072210,2123,53793,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,332.714103,25.583064,19.53736,17.41563,16.41278,15.95257,15.58614,8116,301,3,67,6708188165264660000,-1,0.076411,5958,56213,278
99996,173.757382,36.441603,18.33687,17.30365,17.16037,17.14895,17.14419,4504,301,2,111,2265404129658560000,1,-0.000497,2012,53493,340
99997,205.426531,38.499053,17.50690,15.63152,15.22328,15.04469,15.28668,4382,301,4,97,2257446413900210000,-1,0.004587,2005,53472,62
99998,337.135144,-9.635967,19.33946,17.21436,16.29697,15.86745,15.51556,2576,301,2,105,811847537492257000,-1,0.084538,721,52228,268


Разделите данные на train и test указав random_state = 1, test_size=0.25

In [10]:
# Ваш код тут
X, y = df.iloc[:, :-1], df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# **Задание 1. Функции для оптмизации (4 балла)**

Реализуем несколько полезных функций. Начнем с функции подсчета ошибки дерева - error_rate

$$error\ rate = {\frac{количество\ ошибочно\ предсказанных\ значений}{общее\ количество\ значений}} $$

In [11]:
def error_rate(y_true, y_pred):

    # Ваш код тут

    errors = sum(1 for true, pred in zip(y_true, y_pred) if true != pred)
    error_rate = errors / len(y_true)

    return error_rate

Теперь давайте реализуем подсчет alpha, функция будет определять вес конкретного дерева во всей модели.

$$alpha = 0.5 *{\log_e ({\frac{1 - error\ rate}{error\ rate}})}$$

In [12]:
def alpha(error_rate):

    # Ваш код тут

    if not 0 <= error_rate <= 1:
      raise ValueError('error_rate != [0, 1]')

    if error_rate <= 0:
      return float('inf')
    elif error_rate >= 1:
      return float('-inf')

    # 1e-10 для численной стабильности
    alpha = 0.5 * math.log((1 - error_rate + 1e-10) / (error_rate + 1e-10))

    return alpha

# **Задание 2. Adaboost (12 баллов)**

Теперь мы можем написать наш Adaboost. В нем разрешается использовать DescionTreeClassifier (для них параметры можете выбрать сами). В sklearn можно взять веса для модели и настраивать их, главное не забудьте про их нормализацию, что бы они не были слишком большими

In [13]:
from sklearn.base import clone

class Adaboost():
    def __init__(self, n_estimators=50, learning_rate=0.5, base_classifier=DecisionTreeClassifier()):

        self.n_estimators = n_estimators # количество моделей в adaboost
        self.learning_rate = learning_rate # скорость обучение
        self.base_classifier = base_classifier # модель которую мы используем

        self.alpha = []
        self.classifiers = []

    def train(self, X, y):

       # Задайте начальные веса

        n_samples = len(X)
        weights = np.ones(n_samples) / n_samples

       # Теперь вам нужно сделать цикл, который проходиться по указаному количеству моделей.
       # Модели и alpha добавляйте в self.alpha и self.classifiers
       # При обучении каждой модели в метод fit вам нужно передавать sample_weight=weights
       # При расчете alpha не забудьте его домножить на learning_rate, в лекции он равен 0.5

        for _ in range(self.n_estimators):
            classifier = clone(self.base_classifier)
            classifier.fit(X, y, sample_weight=weights)

            y_pred = classifier.predict(X)

            incorrect = (y_pred != y)
            error = np.sum(weights * incorrect) / np.sum(weights)

            alpha = self.learning_rate * np.log((1 - error) / (error + 1e-10))

            weights *= np.exp(alpha * incorrect)
            weights /= np.sum(weights)

            self.classifiers.append(classifier)
            self.alpha.append(alpha)

           # Весь алгоритм отлично описан в лекции

    def predict(self, X):

        n_samples = len(X)
        y_pred = np.zeros(n_samples)

        for alpha, classifier in zip(self.alpha, self.classifiers):
            y_pred += alpha * classifier.predict(X)

        return np.sign(y_pred)

Протестируйте как работает ваш алгоритм и оцените результат по метрике roc_auc. Она должна быть выше 0.9

In [ ]:
# Ваш код тут

model = Adaboost(n_estimators=50, learning_rate=0.5)
model.train(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print("Train ROC-AUC:", roc_auc_score(y_train, y_pred_train))
print("Test ROC-AUC:", roc_auc_score(y_test, y_pred_test))

# **Задание 3. Catboost (4 балла)**

Теперь мы можем попровать настоящего монстра среди классического ML - Catboost от яндекса. Его не нужно настраивать, хотя и можно(у него тоже есть GridSearch но не из Sklearn). Оцените его резульат по roc_auc.

In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.2 MB/s eta 0:00:00


In [ ]:
# Ваш код тут

from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    eval_metric='AUC',
    verbose=50,
    random_seed=42,
    task_type='CPU'
)

model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred_train_proba = model.predict_proba(X_train)[:, 1]

print(f"\nTrain ROC-AUC CatBoostClassifier: {roc_auc_score(y_train, y_pred_train_proba):.4f}")
print(f"Test ROC-AUC CatBoostClassifier: {roc_auc_score(y_test, y_pred_proba):.4f}")